In [1]:

# # 0. Install Dependencies and Bring in Data
# # !pip install tensorflow tensorflow-gpu pandas matplotlib sklearn
# # !pip list
# # !pip install gradio jinja2

# Importing Packages

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
import gradio as gr
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
import math  
import sklearn.metrics

# Reading Data Set

In [3]:
df = pd.read_csv('./ttrain.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
X = df['comment_text']
y = df[df.columns[2:]].values
p=Precision()
r=Recall()
X

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

# Preprocessing and generating encodings

In [5]:
MAX_FEATURES = 200000 

In [6]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [7]:
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

In [8]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  643,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2506, ...,     0,     0,     0],
       [  425,   440,    70, ...,     0,     0,     0],
       ...,
       [32141,  7329,   383, ...,     0,     0,     0],
       [    5,    12,   533, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [23]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

# Splitting Data Set

In [24]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [11]:
model = tf.keras.models.load_model('toxicitye13.h5')

In [12]:
input_text = vectorizer(["I"])
model.predict(input_text)

array([[2.0411401e-03, 8.9743940e-20, 3.7279769e-05, 5.0133394e-08,
        4.2681268e-06, 2.6238847e-06]], dtype=float32)

# Creating Model

In [13]:
# #  Create Sequential Model
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
# model = Sequential()
# # Create the embedding layer 
# model.add(Embedding(MAX_FEATURES+1, 32))
# # Bidirectional LSTM Layer
# model.add(Bidirectional(LSTM(32, activation='tanh')))
# # Feature extractor Fully connected layers
# model.add(Dense(128, activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# # Final layer
# model.add(Dense(6, activation='sigmoid'))
# # model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='BinaryCrossentropy', optimizer='Adam')
# model.summary()
a=Recall()

In [14]:
def score_comment(comment):
    input_text = vectorizer([comment])
    res=(model.predict(input_text))
    out=''
    l=[]
    a=(res >=0.5).astype(int)
    for i in range(0,len(a[0])):
        if a[0][i]==1:
            l.append(i)
    if len(l)!=0:
        out += 'is_toxic : true\n'
        out += '{}: {}\n'.format('toxic_rate between 1 to 6', len(l))
    else:
        out += 'is_toxic : False\n'
    return(out)

# Creating interface

In [15]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

C:\Users\srini\anaconda3\lib\site-packages\gradio\inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\srini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\srini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


IMPORTANT: You are using gradio version 3.11.0, however version 3.14.0 is available, please upgrade.
--------


In [16]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://f65eb94977c7964a.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


# Evaluating the model

In [25]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    a.update_state(y_true, yhat)
    p.update_state(y_true, yhat)
    r.update_state(y_true, yhat)
    mse = sklearn.metrics.mean_squared_error(y_true, yhat)
    rmse = math.sqrt(mse)  

In [26]:
acc=a.result().numpy()
rec=r.result().numpy()
pre=p.result().numpy()
f1=2*((p.result().numpy()*r.result().numpy())/(p.result().numpy()+r.result().numpy()))

In [28]:
#for detecting and rating toxic text
print(f'Accuracy:{acc}\n\nPrecision:{pre}\n\nRecall:{rec}\n\nF1Score:{f1}\n\nRMSE:{rmse}')

Accuracy:0.9466551542282104

Precision:0.9393419027328491

Recall:0.9366551542282104

F1Score:0.9429843425750732

RMSE:0.00892623808447638


Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [21]:
# 2*((p.result().numpy()*r.result().numpy())/(p.result().numpy()+r.result().numpy()))

In [22]:
# rmse